In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import copy
import math

import matplotlib.image as mpimg
from matplotlib import cm
import matplotlib.animation as ani
from scipy.integrate import odeint
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

import keras

%matplotlib inline

In [3]:
isotopes = Dataset('climatology/xnapa_isotopes.nc', "a")
precipitation = Dataset('climatology/xnapa_precip.nc', "a")
surface_temperature = Dataset('climatology/xnapa_temp.nc', "a")

In [4]:
lat = isotopes.variables["latitude"][:].data[1:-1]
long = isotopes.variables["longitude"][:].data
t = isotopes.variables["t"][:].data

trash = [626, 0]

In [5]:
do18 = []
precip = []
temp = []
for i in range(len(t)):
    do18.append(isotopes.variables["dO18"][i][0].data[1:-1].reshape(71*96))
    precip.append(precipitation.variables["precip"][i][0].data[1:-1].reshape(71*96))
    temp.append(surface_temperature.variables["temp_1"][i][0].data[1:-1].reshape(71*96))
    
do18 = pd.DataFrame(np.delete(np.delete(np.asarray(do18), 0, 0), 626, 0)[:1000])
precip = pd.DataFrame(np.delete(np.delete(np.asarray(precip), 0, 0), 626, 0)[:1000])
temp = pd.DataFrame(np.delete(np.delete(np.asarray(temp), 0, 0), 626, 0)[:1000])

In [6]:
def network_1(X_train, Y_train, X_test, Y_test, n_temp, n_precip, n_y, epochs, batch_size):
    model = Sequential()
    model.add(Dense(2*(n_temp+n_precip), input_dim=n_temp+n_precip, activation='relu'))
    model.add(Dense(2*(n_temp+n_precip), activation='relu'))
    model.add(Dense(n_y, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer="adam")

    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    return model

In [7]:
def network_2(X_train, Y_train, X_test, Y_test, n_temp, n_precip, n_y, epochs, batch_size):
    model = Sequential()
    model.add(Dense(2*(n_temp+n_precip), input_dim=n_temp+n_precip, activation='relu'))
    model.add(Dense(2*(n_temp+n_precip), activation='relu'))
    model.add(Dense(2*(n_temp+n_precip), activation='relu'))
    model.add(Dense(2*(n_temp+n_precip), activation='relu'))
    model.add(Dense(n_y, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer="adam")

    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    return model

In [8]:
def network_3(X_train, Y_train, X_test, Y_test, n_temp, n_precip, n_y, epochs, batch_size):
    model = Sequential()
    model.add(Dense(2*(n_temp+n_precip), input_dim=n_temp+n_precip, activation='relu'))
    model.add(Dense(2*(n_temp+n_precip), activation='relu'))
    model.add(Dense(2*(n_temp+n_precip), activation='linear'))
    model.add(Dense(2*(n_temp+n_precip), activation='linear'))
    model.add(Dense(n_y, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer="adam")

    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    return model

In [9]:
def network_4(X_train, Y_train, X_test, Y_test, n_temp, n_precip, n_y, epochs, batch_size):
    model = Sequential()
    model.add(Dense(2*(n_temp+n_precip), input_dim=n_temp+n_precip, activation='relu'))
    model.add(Dense(3*(n_temp+n_precip), activation='relu'))
    model.add(Dense(3*(n_temp+n_precip), activation='linear'))
    model.add(Dense(n_y, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer="adam")

    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=0, batch_size=batch_size)
    return model

In [10]:
def ann_pca_test(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=1, epochs=50, batch_size=20, 
                 train_proportion=0.9, random=True):
    
    if random:
        random_idx = np.random.permutation(do18.index.values)
    
        temp = temp.reindex(random_idx)
        precip = precip.reindex(random_idx)
        do18 = do18.reindex(random_idx)
    
    #devide training and test data
    do18_train = do18[:int(np.floor(train_proportion*len(do18)))].reset_index(drop=True)
    do18_test = do18[int(np.floor(train_proportion*len(do18))):].reset_index(drop=True)

    temp_train = temp[:int(np.floor(train_proportion*len(temp)))].reset_index(drop=True)
    temp_test = temp[int(np.floor(train_proportion*len(temp))):].reset_index(drop=True)

    precip_train = precip[:int(np.floor(train_proportion*len(precip)))].reset_index(drop=True)
    precip_test = precip[int(np.floor(train_proportion*len(precip))):].reset_index(drop=True)
    
    #define scalers
    scaler_temp = StandardScaler().fit(temp)
    scaler_precip = StandardScaler().fit(precip)
    scaler_do18 = StandardScaler().fit(do18)
    
    #scale data
    do18_train_scaler = scaler_do18.transform(do18_train)
    do18_test_scaler = scaler_do18.transform(do18_test)

    temp_train_scaler = scaler_temp.transform(temp_train)
    temp_test_scaler = scaler_temp.transform(temp_test)

    precip_train_scaler = scaler_precip.transform(precip_train)
    precip_test_scaler = scaler_precip.transform(precip_test)
    
    #define pcas and fit data
    pca_temp = PCA(n_components=n_temp).fit(np.append(temp_train_scaler, temp_test_scaler, axis=0))
    pca_precip = PCA(n_components=n_precip).fit(np.append(precip_train_scaler, precip_test_scaler, axis=0))
    pca_do18 = PCA(n_components=n_y).fit(np.append(do18_train_scaler, do18_test_scaler, axis=0))
    
    #transform data
    do18_train_pc = pca_do18.transform(do18_train_scaler)
    do18_test_pc = pca_do18.transform(do18_test_scaler)

    temp_train_pc = pca_temp.transform(temp_train_scaler)
    temp_test_pc = pca_temp.transform(temp_test_scaler)

    precip_train_pc = pca_precip.transform(precip_train_scaler)
    precip_test_pc = pca_precip.transform(precip_test_scaler)
    
    
    X_train = np.append(temp_train_pc, precip_train_pc, axis=1)+100
    X_test = np.append(temp_test_pc, precip_test_pc, axis=1)+100

    Y_train = do18_train_pc+100
    Y_test = do18_test_pc+100

    Y = pd.DataFrame(np.append(Y_train, Y_test, axis=0))-100
    
    #####################################################################
    #train model
    if network==1:
        model = network_1(X_train, Y_train, X_test, Y_test, n_temp, n_precip, n_y, epochs, batch_size)
    elif network==2:
        model = network_2(X_train, Y_train, X_test, Y_test, n_temp, n_precip, n_y, epochs, batch_size)
    elif network==3:
        model = network_3(X_train, Y_train, X_test, Y_test, n_temp, n_precip, n_y, epochs, batch_size)
    elif network==4:
        model = network_4(X_train, Y_train, X_test, Y_test, n_temp, n_precip, n_y, epochs, batch_size)
    
    ####################################################################
    
    #predict test data
    pred_pc = pd.DataFrame(model.predict(X_test)-100)
    pred_norm = pd.DataFrame(pca_do18.inverse_transform(pred_pc))
    pred = pd.DataFrame(scaler_do18.inverse_transform(pred_norm))
    
    
    #return pca and final data
    return pred_pc, pred_norm, pred, do18_test_pc, pd.DataFrame(do18_test_scaler), do18_test

In [11]:
def test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=1, epochs=50, batch_size=20, runs=10,
                 train_proportion=0.9, random=True):
    
    pred_pc, pred_norm, pred, Y_test, do18_test_scaler, do18_test = ann_pca_test(
            temp, precip, do18, n_temp, n_precip, n_y, 
            network, epochs, batch_size, 
            train_proportion, random)
    
    pred_list = copy.deepcopy(pred)
    test_list = copy.deepcopy(do18_test)
    
    for i in range(runs):
        pred_pc, pred_norm, pred, Y_test, do18_test_scaler, do18_test = ann_pca_test(
            temp, precip, do18, n_temp, n_precip, n_y, 
            network, epochs, batch_size, 
            train_proportion, random)
        pred_list = pred_list.append(pred)
        test_list = test_list.append(do18_test)
    pred_list = pred_list.reset_index(drop=True)
    test_list = test_list.reset_index(drop=True)
    
    rmse_geolist = []
    for pixel in test_list.columns.values:
        rmse_pixel = mean_squared_error(pred_list[pixel].values, test_list[pixel].values, squared=False)
        rmse_geolist.append(rmse_pixel)
    df_rmse_geolist = pd.DataFrame(np.asarray(rmse_geolist).reshape(71,96))
    
    return df_rmse_geolist

In [12]:
weights = np.cos(np.deg2rad(lat))
weights = weights/weights.sum()
weights = weights[:, np.newaxis] * np.ones_like(do18.mean().values.reshape(71,96))

def weighted_avg_std(values, weights):
    average = np.average(values, weights=weights)
    variance = (((values - average)**2)*weights).sum()/(weights.sum() - (weights**2).sum()/weights.sum())
    return (average, math.sqrt(variance))

## Test different neural networks 

In [55]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=1, epochs=50, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

In [64]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=1, epochs=200, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.86 +- 0.95 permil


In [65]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=1, epochs=400, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.85 +- 0.95 permil


In [66]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=50, 
                 network=1, epochs=200, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.88 +- 0.97 permil


In [67]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=50, 
                 network=1, epochs=400, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.95 +- 1.00 permil


In [68]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=50, 
                 network=1, epochs=400, batch_size=10, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.83 +- 0.94 permil


In [69]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=2, epochs=200, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.87 +- 0.96 permil


In [70]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=2, epochs=400, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.93 +- 0.99 permil


In [71]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=2, epochs=200, batch_size=10, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.82 +- 0.95 permil


In [72]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=3, epochs=200, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.85 +- 0.96 permil


In [13]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=3, epochs=200, batch_size=10, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))


0.89 +- 0.97 permil


In [14]:
df = test_network(temp, precip, do18, n_temp=20, n_precip=20, n_y=20, 
                 network=4, epochs=200, batch_size=20, runs=10)

print(r"{:.2f} +- {:.2f} permil".format(weighted_avg_std(df.values, weights)[0], weighted_avg_std(df.values, weights)[1]))

0.89 +- 0.97 permil
